# Clasificación de Alzheimer mediante MRI
Red neuronal para clasificar imágenes de MRI en 4 clases diferentes de Alzheimer:
•	No Demencia
•	Demencia Muy Leve
•	Demencia Leve
•	Demencia Moderada

El modelo tiene como objetivo clasificar imágenes de MRI del cerebro a través de una red neuronal convolucional, útil para detectar cuatro etapas de demencia en el cribado de la enfermedad de Alzheimer.


In [1]:
# Libraries importation
import os # operating system interfaces
from os.path import join
from glob import glob
import numpy as np # scientific computing
import pandas as pd # data analysis
import tensorflow as tf
import matplotlib.pyplot as plt # plots
import PIL # image files manipulation
import random

# callback to reduce learning rate
from tensorflow.keras.callbacks import ReduceLROnPlateau

# opencv to handle images (https://opencv.org/releases/)
import cv2 as cv

# print metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, \
    confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Number of replicas: 1
2.18.0


##### **Valores constantes**

En esta parte, se establecen cuatro parámetros:

- **AUTOTUNE:** controla la optimización de la potencia de cálculo.
- **TAMAÑO DEL LOTE (BATCH SIZE):** define el subconjunto de datos sobre el cual calcular el gradiente. Después de algunas ejecuciones, decidimos establecer el hiperparámetro en 32, también dependiendo de si el TPU está habilitado o no.
- **TAMAÑO DE LA IMAGEN (IMAGE SIZE):** define el tamaño de las imágenes que ingresan a la primera capa del modelo.
- **ÉPOCAS (EPOCHS):** establece el número máximo de épocas en las que entrenar el modelo.


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # automatic paralleling computing
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
IMAGE_SIZE = [176, 208]
EPOCHS = 50 # reduced from 100 because of no significant improvement after 50th epoch

In [ ]:
import pandas as pd
from glob import glob
from os.path import join
import os

# Obtener la ruta actual del script
current_dir = os.getcwd()
print("Directorio actual:", current_dir)

# Definir la ruta base - ajusta esto según donde esté tu notebook en relación a la carpeta AlzCare-AI
#data_folder = "./AlzCare-AI/data/tomography_images"  # Si estás en la raíz del proyecto
# O alternativamente:
data_folder = "data/tomography_images"  # Si estás dentro de la carpeta AlzCare-AI

# Verificar si las carpetas existen
print("\nVerificando rutas:")
print(f"Ruta completa: {os.path.abspath(data_folder)}")
print(f"¿Existe la carpeta base? {os.path.exists(data_folder)}")
for category in ["MildDemented", "ModerateDemented", "NonDemented", "VeryMildDemented"]:
    full_path = join(data_folder, category)
    print(f"¿Existe {category}? {os.path.exists(full_path)}")

# Resto del código igual, pero añadimos verificación de patrones de archivo
for category in ["MildDemented", "ModerateDemented", "NonDemented", "VeryMildDemented"]:
    pattern = join(data_folder, category, "*")
    files = glob(pattern)
    print(f"\nArchivos encontrados en {category}: {len(files)}")
    if len(files) > 0:
        print(f"Primer archivo encontrado: {files[0]}")

## Continúa con el resto del código original...

Directorio actual: c:\F5_Proyectos\AlzCare-AI\AlzCare-AI\AlzCare-AI\notebooks

Verificando rutas:
Ruta completa: c:\F5_Proyectos\AlzCare-AI\AlzCare-AI\AlzCare-AI\notebooks\AlzCare-AI\data\tomography_images
¿Existe la carpeta base? False
¿Existe MildDemented? False
¿Existe ModerateDemented? False
¿Existe NonDemented? False
¿Existe VeryMildDemented? False

Archivos encontrados en MildDemented: 0

Archivos encontrados en ModerateDemented: 0

Archivos encontrados en NonDemented: 0

Archivos encontrados en VeryMildDemented: 0


Cargar imágenes por categorías

In [5]:
import pandas as pd
from glob import glob
from os.path import join

# Definir la nueva ruta base
data_folder = "../AlzCare-AI/data/tomography_images/"

## Imágenes NonDemented
df_NonDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "NonDemented", "*"))),
    "y": 0,
    "class": "NonDementia"
})
shuffled_non = df_NonDemented_imgs.sample(frac=1)
testsize_non = int(0.2 * len(shuffled_non))

# Test
non_test = shuffled_non[:testsize_non]

# Train and validation
non_trainval = shuffled_non[testsize_non:]
trainsize_non = int(0.8 * len(non_trainval))

# Train
shuffled_non_train = non_trainval[:trainsize_non]

# Validation
shuffled_non_val = non_trainval[trainsize_non:]

## Imágenes VeryMildDemented
df_VeryMildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "VeryMildDemented", "*"))),
    "y": 1,
    "class": "VeryMildDementia"
})
shuffled_verymild = df_VeryMildDemented_imgs.sample(frac=1)
testsize_verymild = int(0.2 * len(shuffled_verymild))

# Test
verymild_test = shuffled_verymild[:testsize_verymild]

# Train and validation
verymild_trainval = shuffled_verymild[testsize_verymild:]
trainsize_verymild = int(0.8 * len(verymild_trainval))

# Train
shuffled_verymild_train = verymild_trainval[:trainsize_verymild]

# Validation
shuffled_verymild_val = verymild_trainval[trainsize_verymild:]

## Imágenes MildDemented
df_MildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "MildDemented", "*"))),
    "y": 2,
    "class": "MildDementia"
})
shuffled_mild = df_MildDemented_imgs.sample(frac=1)
testsize_mild = int(0.2 * len(shuffled_mild))

# Test
mild_test = shuffled_mild[:testsize_mild]

# Train and validation
mild_trainval = shuffled_mild[testsize_mild:]
trainsize_mild = int(0.8 * len(mild_trainval))

# Train
shuffled_mild_train = mild_trainval[:trainsize_mild]

# Validation
shuffled_mild_val = mild_trainval[trainsize_mild:]

## Imágenes ModerateDemented
df_ModerateDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "ModerateDemented", "*"))),
    "y": 3,
    "class": "ModerateDementia"
})
shuffled_moderate = df_ModerateDemented_imgs.sample(frac=1)
testsize_moderate = int(0.2 * len(shuffled_moderate))

# Test
moderate_test = shuffled_moderate[:testsize_moderate]

# Train and validation
moderate_trainval = shuffled_moderate[testsize_moderate:]
trainsize_moderate = int(0.8 * len(moderate_trainval))

# Train
shuffled_moderate_train = moderate_trainval[:trainsize_moderate]

# Validation
shuffled_moderate_val = moderate_trainval[trainsize_moderate:]

## Número de imágenes
print("TOTAL:")
print("# de imágenes NonDemented Alzheimer =", len(shuffled_non))
print("# de imágenes VeryMildDemented Alzheimer =", len(shuffled_verymild))
print("# de imágenes MildDemented Alzheimer =", len(shuffled_mild))
print("# de imágenes ModerateDemented Alzheimer =", len(shuffled_moderate))
print("------------")
print("\nTest:")
print("# de imágenes NonDemented Alzheimer =", len(non_test))
print("# de imágenes VeryMildDemented Alzheimer =", len(verymild_test))
print("# de imágenes MildDemented Alzheimer =", len(mild_test))
print("# de imágenes ModerateDemented Alzheimer =", len(moderate_test))
print("------------")
print("\nEntrenamiento:")
print("# de imágenes NonDemented Alzheimer =", len(shuffled_non_train))
print("# de imágenes VeryMildDemented Alzheimer =", len(shuffled_verymild_train))
print("# de imágenes MildDemented Alzheimer =", len(shuffled_mild_train))
print("# de imágenes ModerateDemented Alzheimer =", len(shuffled_moderate_train))
print("------------")
print("\nValidación:")
print("# de imágenes NonDemented Alzheimer =", len(shuffled_non_val))
print("# de imágenes VeryMildDemented Alzheimer =", len(shuffled_verymild_val))
print("# de imágenes MildDemented Alzheimer =", len(shuffled_mild_val))
print("# de imágenes ModerateDemented Alzheimer =", len(shuffled_moderate_val))

TOTAL:
# de imágenes NonDemented Alzheimer = 0
# de imágenes VeryMildDemented Alzheimer = 0
# de imágenes MildDemented Alzheimer = 0
# de imágenes ModerateDemented Alzheimer = 0
------------

Test:
# de imágenes NonDemented Alzheimer = 0
# de imágenes VeryMildDemented Alzheimer = 0
# de imágenes MildDemented Alzheimer = 0
# de imágenes ModerateDemented Alzheimer = 0
------------

Entrenamiento:
# de imágenes NonDemented Alzheimer = 0
# de imágenes VeryMildDemented Alzheimer = 0
# de imágenes MildDemented Alzheimer = 0
# de imágenes ModerateDemented Alzheimer = 0
------------

Validación:
# de imágenes NonDemented Alzheimer = 0
# de imágenes VeryMildDemented Alzheimer = 0
# de imágenes MildDemented Alzheimer = 0
# de imágenes ModerateDemented Alzheimer = 0
